In [1]:
import numpy as np
from scipy.special import factorial2 as fact2
import basis_set_exchange

In [2]:
bs_dict = basis_set_exchange.get_basis('6-31G*',elements=[6], fmt='gaussian94', header=False)
help(bs_dict)

No Python documentation found for 'C     0\nS   6   1.00\n      0.3047524880D+04       0.1834737132D-02\n      0.4573695180D+03       0.1403732281D-01\n      0.1039486850D+03       0.6884262226D-01\n      0.2921015530D+02       0.2321844432D+00\n      0.9286662960D+01       0.4679413484D+00\n      0.3163926960D+01       0.3623119853D+00\nSP   3   1.00\n      0.7868272350D+01      -0.1193324198D+00       0.6899906659D-01\n      0.1881288540D+01      -0.1608541517D+00       0.3164239610D+00\n      0.5442492580D+00       0.1143456438D+01       0.7443082909D+00\nSP   1   1.00\n      0.1687144782D+00       0.1000000000D+01       0.1000000000D+01\nD   1   1.00\n      0.8000000000D+00       1.0000000\n****'.
Use help() to get the interactive help utility.
Use help(str) for help on the str class.



In [3]:
def E(i,j,t,Qx,a,b):
    ''' Recursive definition of Hermite Gaussian coefficients.
        Returns a float.
        a: orbital exponent on Gaussian 'a' (e.g. alpha in the text)
        b: orbital exponent on Gaussian 'b' (e.g. beta in the text)
        i,j: orbital angular momentum number on Gaussian 'a' and 'b'
        t: number nodes in Hermite (depends on type of integral, 
           e.g. always zero for overlap integrals)
        Qx: distance between origins of Gaussian 'a' and 'b'
    '''
    p = a + b
    q = a*b/p
    if (t < 0) or (t > (i + j)):
        # out of bounds for t  
        return 0.0
    elif i == j == t == 0:
        # base case
        return np.exp(-q*Qx*Qx) # K_AB
    elif j == 0:
        # decrement index i
        return (1/(2*p))*E(i-1,j,t-1,Qx,a,b) - \
               (q*Qx/a)*E(i-1,j,t,Qx,a,b)    + \
               (t+1)*E(i-1,j,t+1,Qx,a,b)
    else:
        # decrement index j
        return (1/(2*p))*E(i,j-1,t-1,Qx,a,b) + \
               (q*Qx/b)*E(i,j-1,t,Qx,a,b)    + \
               (t+1)*E(i,j-1,t+1,Qx,a,b)

In [4]:
def overlap(a,lmn1,A,b,lmn2,B):
    ''' Evaluates overlap integral between two Gaussians
        Returns a float.
        a:    orbital exponent on Gaussian 'a' (e.g. alpha in the text)
        b:    orbital exponent on Gaussian 'b' (e.g. beta in the text)
        lmn1: int tuple containing orbital angular momentum (e.g. (1,0,0))
              for Gaussian 'a'
        lmn2: int tuple containing orbital angular momentum for Gaussian 'b'
        A:    list containing origin of Gaussian 'a', e.g. [1.0, 2.0, 0.0]
        B:    list containing origin of Gaussian 'b'
    '''
    l1,m1,n1 = lmn1 # shell angular momentum on Gaussian 'a'
    l2,m2,n2 = lmn2 # shell angular momentum on Gaussian 'b'
    S1 = E(l1,l2,0,A[0]-B[0],a,b) # X
    S2 = E(m1,m2,0,A[1]-B[1],a,b) # Y
    S3 = E(n1,n2,0,A[2]-B[2],a,b) # Z
    return S1*S2*S3*np.power(np.pi/(a+b),1.5) 

In [5]:
def S(a,b):
    '''Evaluates overlap between two contracted Gaussians
       Returns float.
       Arguments:
       a: contracted Gaussian 'a', BasisFunction object
       b: contracted Gaussian 'b', BasisFunction object
    '''
    s = 0.0
    for ia, ca in enumerate(a.coefs):
        for ib, cb in enumerate(b.coefs):
            s += a.norm[ia]*b.norm[ib]*ca*cb*\
                     overlap(a.exps[ia],a.shell,a.origin,
                     b.exps[ib],b.shell,b.origin)
    return s

In [6]:
class BasisFunction(object):
    ''' A class that contains all our basis function data
        Attributes:
        origin: array/list containing the coordinates of the Gaussian origin
        shell:  tuple of angular momentum
        exps:   list of primitive Gaussian exponents
        coefs:  list of primitive Gaussian coefficients
        norm:   list of normalization factors for Gaussian primitives
    '''
    def __init__(self,origin=[0.0,0.0,0.0],shell=(0,0,0),exps=[],coefs=[]):
        self.origin = np.asarray(origin)
        self.shell = shell
        self.exps  = exps
        self.coefs = coefs
        self.comp_norm()
        self.normalize()
    def comp_norm(self):
        #norm are prefactoprs for normalize each gaussian individually
        l,m,n = self.shell
        self.norm = np.sqrt(np.power(2,2*(l+m+n)+1.5)*
                        np.power(self.exps,l+m+n+1.5)/
                        fact2(2*l-1)/fact2(2*m-1)/
                        fact2(2*n-1)/np.power(np.pi,1.5))
        
    def normalize(self):
        ''' Routine to normalize the basis functions, in case they
            do not integrate to unity.
        '''
        l,m,n = self.shell
        L = l+m+n
        # self.norm is a list of length equal to number primitives
        # normalize primitives first (PGBFs)
        self.comp_norm()
        # now normalize the contracted basis functions (CGBFs)
        # Eq. 1.44 of Valeev integral whitepaper
        prefactor = np.power(np.pi,1.5)*\
            fact2(2*l - 1)*fact2(2*m - 1)*fact2(2*n - 1)/np.power(2.0,L)
        N = 0.0
        num_exps = len(self.exps)
        for ia in range(num_exps):
            for ib in range(num_exps):
                N+=self.norm[ia]*self.norm[ib]*self.coefs[ia]*self.coefs[ib]/np.power(self.exps[ia] + self.exps[ib],L+1.5)
        print(N)
        N *= prefactor
        print(N)

        N = np.power(N,-0.5)
        for ia in range(num_exps):
            self.coefs[ia] *= N

In [7]:
myOrigin = [1.0, 2.0, 3.0]
myShell  = (0,0,0) #p-orbitals would be (1,0,0) or (0,1,0) or (0,0,1), etc.
myExps   = [3.42525091, 0.62391373, 0.16885540] 
myCoefs  = [0.15432897, 0.53532814, 0.44463454]
a = BasisFunction(origin=myOrigin,shell=myShell,exps=myExps,coefs=myCoefs)

0.17958712048909195
0.9999999908898


In [8]:
myExps=[4563.240,682.0240,154.9730,44.45530,13.02900,1.827730]
myCoefs=[0.00196665, 0.0152306,0.0761269,0.2608010, 0.6164620,0.2210060]
myShell  = (0,0,0)
a = BasisFunction(origin=myOrigin,shell=myShell,exps=myExps,coefs=myCoefs)
a.norm

0.17958707656790368
0.9999997463222177


array([395.69911167,  95.1173704 ,  31.30416489,  12.27023135,
         4.88757467,   1.12032525])

In [8]:
Saa=0
for i in range(len(myCoefs)):
    for j in range(len(myCoefs)):
        Saa+=a.norm[i]*a.norm[j]*myCoefs[i]*myCoefs[j]*(np.pi/(myExps[i]+myExps[j]))**1.5
Saa

1.0000000000000002

[0.1543289707029839, 0.5353281424384732, 0.44463454202535485]